## Import subtitles

In [58]:
import glob
import os
import sys

In [77]:
seasons = [1, 6, 7, 10]
seasons = [str(season) for season in seasons]

In [78]:
BASE_PATH = './friends-season-1/'
idcs = [str(_).zfill(2) for _ in range(1, 25)]
subtitles_data = []

for season in seasons:
    path = './data/friends-season-' + season + '/'
    for file in os.listdir(path):
        f = open(path + file, 'rb')
        content = f.read().decode('utf8', 'ignore')
        subtitles_data.append(content)

## Extracting only lines (대사) from the files

In [80]:
def extract_dialogue(file_content):
    dialogue = []
    lines = file_content.split('\r\n\r\n')
    for line in lines:
        dialogue.extend(line.split('\r\n')[2:])
    return dialogue

In [81]:
subtitles_lines = []
for content in subtitles_data:
    subtitles_lines.append(extract_dialogue(content))

In [95]:
from nltk.tokenize import word_tokenize

sentences = []
for subtitle_lines in subtitles_lines:
    # first three sentences and the last one are not to be used
    for line in subtitle_lines[3:-1]:
        sentences.append(word_tokenize(line))

In [96]:
len(sentences)

44902

## Word2Vec model

In [86]:
from gensim.models import Word2Vec
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

In [87]:
# example
# from nltk.corpus import movie_reviews
# sentences = [list(s) for s in movie_reviews.sents()]

In [97]:
%%time
model = Word2Vec(sentences)

CPU times: user 1.2 s, sys: 4.28 ms, total: 1.2 s
Wall time: 487 ms


In [98]:
model.init_sims(replace=True)

In [99]:
model.most_similar('Ross')

[('Joey', 0.9551360011100769),
 ('Chandler', 0.948361873626709),
 ('Phoebe', 0.9472805261611938),
 ('Rachel', 0.9349650740623474),
 ('Pheebs', 0.9262493848800659),
 ('Rach', 0.9178129434585571),
 ('Yes', 0.9110141396522522),
 ('Hey', 0.9021619558334351),
 ('Monica', 0.8921801447868347),
 ('hey', 0.8902468085289001)]